In [17]:
# Import pyspark.pandas
import pyspark.pandas as ps

In [18]:
crime_df = ps.read_csv("data/crime.csv")
crime_df.head()

C:\DISTR\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,None,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,None,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,None,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,None,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,None,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [19]:
crime_df.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 319073 entries, 0 to 319072
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   INCIDENT_NUMBER      319073 non-null  object        
 1   OFFENSE_CODE         319073 non-null  int32         
 2   OFFENSE_CODE_GROUP   319073 non-null  object        
 3   OFFENSE_DESCRIPTION  319073 non-null  object        
 4   DISTRICT             317308 non-null  object        
 5   REPORTING_AREA       319073 non-null  object        
 6   SHOOTING             1019 non-null    object        
 7   OCCURRED_ON_DATE     319073 non-null  datetime64[ns]
 8   YEAR                 319073 non-null  int32         
 9   MONTH                319073 non-null  int32         
 10  DAY_OF_WEEK          319073 non-null  object        
 11  HOUR                 319073 non-null  int32         
 12  UCR_PART             318983 non-null  object        
 13  STREET     

In [20]:
crime_df_wo_dup = crime_df.drop_duplicates()
crime_df_wo_dup.shape

(319050, 17)

In [21]:
codes_df = ps.read_csv("data/offense_codes.csv")
codes_df.head()

C:\DISTR\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,CODE,NAME
0,612,LARCENY PURSE SNATCH - NO FORCE
1,613,LARCENY SHOPLIFTING
2,615,LARCENY THEFT OF MV PARTS & ACCESSORIES
3,1731,INCEST
4,3111,LICENSE PREMISE VIOLATION


In [22]:
codes_df["crime_type"] = codes_df["NAME"].apply(lambda x: x.split(" - ")[0])
code_df_gr = codes_df.groupby("CODE", as_index=False).aggregate({
    "crime_type" : "first"
})
code_df_gr.head()

,CODE,crime_type
0,111,"MURDER, NON-NEGLIGIENT MANSLAUGHTER"
1,112,KILLING OF FELON BY POLICE
2,113,KILLING OF FELON BY CITIZEN
3,114,KILLING OF POLICE BY FELON
4,121,MANSLAUGHTER


In [23]:
df_merged_codes = ps.merge(
    crime_df_wo_dup,
    code_df_gr,
    how="left",
    left_on="OFFENSE_CODE",
    right_on="CODE"
)
df_merged_codes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,CODE,crime_type
0,I080542626-00,1849,Drug Violation,"DRUGS - POSS CLASS B - COCAINE, ETC.",A1,111,None,2015-08-12 12:00:00,2015,8,Wednesday,12,Part Two,BOYLSTON ST,42.352312,-71.063705,"(42.35231190, -71.06370510)",1849,DRUGS
1,I100340225-00,3125,Warrant Arrests,WARRANT ARREST,A1,77,None,2015-07-27 10:47:00,2015,7,Monday,10,Part Three,BOWDOIN SQ,42.361645,-71.062299,"(42.36164502, -71.06229949)",3125,WARRANT ARREST
2,I110694557-00,1605,Prostitution,PROSTITUTE - COMMON NIGHTWALKER,B3,436,None,2016-01-22 09:45:00,2016,1,Friday,9,Part Two,WITHINGTON ST,42.288767,-71.072897,"(42.28876736, -71.07289694)",1605,PROSTITUTION
3,I120201612-00,3125,Warrant Arrests,WARRANT ARREST,B2,292,None,2017-01-20 02:00:00,2017,1,Friday,2,Part Three,JOHN ELIOT SQ,42.330013,-71.091179,"(42.33001318, -71.09117863)",3125,WARRANT ARREST
4,I130041200-00,540,Commercial Burglary,B&E NON-RESIDENCE DAY - FORCIBLE,B3,428,None,2015-12-07 11:38:00,2015,12,Monday,11,Part One,DIXWELL ST,42.314334,-71.097914,"(42.31433402, -71.09791405)",540,BURGLARY


In [24]:
df_grouped = df_merged_codes.groupby(["DISTRICT"], as_index=False).agg({
    "INCIDENT_NUMBER": "count",
    "Lat": "mean",
    "Long": "mean"
}).rename(columns={"DISTRICT":"district", "INCIDENT_NUMBER":"crimes_total", "Lat":"lat", "Long":"lng"})
df_grouped.head()

,district,crimes_total,lat,lng
0,C6,23457,42.212105,-70.855582
1,B2,49940,42.316003,-71.075698
2,C11,42528,42.292638,-71.051260
3,E13,17534,42.309803,-71.098005
4,B3,35441,42.283059,-71.078949


In [25]:
df_distr_type = df_merged_codes.groupby(["DISTRICT", "crime_type"])["INCIDENT_NUMBER"].count().reset_index()
df_distr_type[df_distr_type["DISTRICT"] == "A15"].sort_values(by = 'INCIDENT_NUMBER', ascending = False).head(3)

,DISTRICT,crime_type,INCIDENT_NUMBER
141,A15,M/V ACCIDENT,505
215,A15,INVESTIGATE PERSON,464
533,A15,M/V,430


In [26]:
df_top_crimes = df_distr_type.groupby(["DISTRICT"]).apply(
    lambda x: x.nlargest(3,["INCIDENT_NUMBER"])["crime_type"].str.cat(sep=", ")
).reset_index().rename(
    columns={"DISTRICT":"district", 0:"frequent_crime_types"}
)
df_top_crimes.head()

C:\DISTR\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `grouby.apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,district,frequent_crime_types
0,A1,"PROPERTY, ASSAULT SIMPLE, DRUGS"
1,A15,"M/V ACCIDENT, INVESTIGATE PERSON, M/V"
2,A7,"SICK/INJURED/MEDICAL, DRUGS, INVESTIGATE PERSON"
3,B2,"M/V, M/V ACCIDENT, VERBAL DISPUTE"
4,B3,"VERBAL DISPUTE, INVESTIGATE PERSON, MISSING PE..."


In [27]:
df_distr_month = df_merged_codes.groupby(["DISTRICT", "YEAR", "MONTH"])["INCIDENT_NUMBER"].count().reset_index()[["DISTRICT", "INCIDENT_NUMBER"]]
df_distr_month.sort_values(by="DISTRICT").head()

,DISTRICT,INCIDENT_NUMBER
20,A1,790
29,A1,1000
32,A1,844
54,A1,904
59,A1,965


In [28]:
df_med_month = df_distr_month.groupby(["DISTRICT"]).median().reset_index().rename(columns={"DISTRICT":"district", "INCIDENT_NUMBER":"crimes_monthly"})
df_med_month.head()

,district,crimes_monthly
0,C6,592
1,B2,1298
2,C11,1115
3,E13,445
4,B3,907


In [29]:
df_merged = ps.merge(
    df_grouped,
    df_top_crimes,
    on="district",
)
df_full = ps.merge(
    df_merged,
    df_med_month,
    on="district",
)
df_full.head()

,district,crimes_total,lat,lng,frequent_crime_types,crimes_monthly
0,A1,35710,42.331225,-71.019909,"PROPERTY, ASSAULT SIMPLE, DRUGS",904
1,A15,6505,42.179155,-70.744725,"M/V ACCIDENT, INVESTIGATE PERSON, M/V",160
2,A7,13544,42.360703,-71.003948,"SICK/INJURED/MEDICAL, DRUGS, INVESTIGATE PERSON",344
3,B2,49940,42.316003,-71.075698,"M/V, M/V ACCIDENT, VERBAL DISPUTE",1298
4,B3,35441,42.283059,-71.078949,"VERBAL DISPUTE, INVESTIGATE PERSON, MISSING PE...",907


In [30]:
df_full.to_parquet("data/result_parquet", index=False)

C:\DISTR\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_parquet`, the existing index is lost when converting to Parquet.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
C:\DISTR\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [31]:
df_full.to_csv("data/result_csv", index=False)